# ⚡ Notebook 05: Liquid Clustering - Optimización Avanzada

## Objetivo
Explorar y demostrar **Liquid Clustering** en Azure Databricks:
- Qué es y cómo funciona
- Comparación con Partitioning y Z-Ordering tradicionales
- Benchmarks de rendimiento
- Mejores prácticas y cuándo usarlo

## ¿Qué es Liquid Clustering?

**Liquid Clustering** es una técnica de optimización de datos en Delta Lake que:

| Característica | Descripción |
|----------------|-------------|
| **Flexible** | Puedes cambiar las columnas de clustering sin reescribir toda la tabla |
| **Incremental** | Solo optimiza los datos nuevos/modificados |
| **Automático** | Se integra con `OPTIMIZE` para mantenimiento continuo |
| **Sin particiones** | Elimina la necesidad de elegir particiones estáticas |

```
┌──────────────────────────────────────────────────────────────────┐
│                    LIQUID CLUSTERING                             │
├──────────────────────────────────────────────────────────────────┤
│                                                                  │
│   Antes (Partitioning):          Después (Liquid Clustering):   │
│   ┌─────────────────┐            ┌─────────────────┐            │
│   │ /date=2024-01/  │            │ Datos organizados            │
│   │   /region=US/   │    ──►     │ dinámicamente por            │
│   │     file1.parquet│            │ CLUSTER BY (col1, col2)     │
│   └─────────────────┘            │                              │
│   ⚠️ Estructura rígida           │ ✅ Flexible y eficiente      │
│   ⚠️ Small files problem         │ ✅ Sin small files           │
│   ⚠️ Difícil de cambiar          │ ✅ Fácil de modificar        │
│                                   └─────────────────┘            │
└──────────────────────────────────────────────────────────────────┘
```

## 1. Configuración y Verificación de Compatibilidad

In [ ]:
# Verificar versión de Databricks Runtime
import os

# Obtener información del cluster
spark_version = spark.version
dbr_version = spark.conf.get("spark.databricks.clusterUsageTags.sparkVersion", "Unknown")

print("🔍 VERIFICACIÓN DE COMPATIBILIDAD")
print("="*50)
print(f"Spark Version: {spark_version}")
print(f"Databricks Runtime: {dbr_version}")

# Liquid Clustering requiere DBR 13.3+ para creación y DBR 15.2+ para clustering manual
print("\n📋 Requisitos de Liquid Clustering:")
print("   • DBR 13.3+: Crear tablas con CLUSTER BY")
print("   • DBR 14.1+: Optimización mejorada")
print("   • DBR 15.2+: Clustering manual en Standard tier")
print("\n✅ Azure Databricks Standard tier: SOPORTADO (DBR 15.2+)")

In [ ]:
# Configuración
DATABASE_NAME = "financial_lakehouse"
spark.sql(f"USE {DATABASE_NAME}")

# Habilitar optimizaciones
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.databricks.delta.autoCompact.enabled", "true")

print(f"✅ Base de datos: {DATABASE_NAME}")

---
## 2. Comparación: Partitioning vs Z-Order vs Liquid Clustering

Vamos a crear tres versiones de la misma tabla para comparar:

In [ ]:
# Crear datos de prueba más grandes para benchmarks
from pyspark.sql.functions import *
from pyspark.sql.types import *
import random

print("🔄 Generando datos de prueba para benchmarks...")

# Generar 1 millón de transacciones para pruebas de rendimiento
num_records = 1000000

# Crear DataFrame con datos sintéticos
df_benchmark = (
    spark.range(0, num_records)
    .withColumn("transaction_id", concat(lit("TXN-"), format_string("%010d", col("id"))))
    .withColumn("fecha", date_sub(current_date(), (rand() * 365).cast("int")))
    .withColumn("cliente_id", concat(lit("CLI-"), format_string("%06d", (rand() * 10000).cast("int"))))
    .withColumn("cuenta_id", concat(lit("CTA-"), format_string("%08d", (rand() * 15000).cast("int"))))
    .withColumn("tipo_transaccion", 
        when(rand() < 0.3, "DEPOSITO")
        .when(rand() < 0.5, "RETIRO")
        .when(rand() < 0.7, "TRANSFERENCIA")
        .otherwise("PAGO")
    )
    .withColumn("monto", round(rand() * 10000 + 10, 2))
    .withColumn("region", 
        when(rand() < 0.25, "NORTE")
        .when(rand() < 0.50, "SUR")
        .when(rand() < 0.75, "ESTE")
        .otherwise("OESTE")
    )
    .withColumn("sucursal", concat(lit("SUC-"), format_string("%03d", (rand() * 100).cast("int"))))
    .withColumn("canal", 
        when(rand() < 0.4, "APP_MOVIL")
        .when(rand() < 0.7, "WEB")
        .when(rand() < 0.9, "ATM")
        .otherwise("SUCURSAL")
    )
    .withColumn("estado", 
        when(rand() < 0.95, "COMPLETADA")
        .when(rand() < 0.98, "PENDIENTE")
        .otherwise("FALLIDA")
    )
    .drop("id")
)

# Cache para reutilizar
df_benchmark.cache()
count = df_benchmark.count()

print(f"✅ Generados {count:,} registros para benchmarks")
df_benchmark.printSchema()

### 2.1 Tabla con Partitioning Tradicional

In [ ]:
%%time
# Opción 1: Partitioning tradicional
print("📁 Creando tabla con PARTITIONING tradicional...")

# Eliminar si existe
spark.sql("DROP TABLE IF EXISTS benchmark_partitioned")

# Crear tabla particionada por fecha y región
(
    df_benchmark
    .write
    .format("delta")
    .mode("overwrite")
    .partitionBy("fecha", "region")
    .saveAsTable("benchmark_partitioned")
)

print("✅ Tabla benchmark_partitioned creada")

### 2.2 Tabla con Z-Ordering

In [ ]:
%%time
# Opción 2: Sin particiones pero con Z-Order
print("📁 Creando tabla con Z-ORDER...")

spark.sql("DROP TABLE IF EXISTS benchmark_zorder")

# Crear tabla sin particiones
(
    df_benchmark
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("benchmark_zorder")
)

# Aplicar Z-Order después
spark.sql("OPTIMIZE benchmark_zorder ZORDER BY (fecha, cliente_id, region)")

print("✅ Tabla benchmark_zorder creada y optimizada")

### 2.3 Tabla con Liquid Clustering

In [ ]:
%%time
# Opción 3: Liquid Clustering
print("📁 Creando tabla con LIQUID CLUSTERING...")

spark.sql("DROP TABLE IF EXISTS benchmark_liquid")

# Crear tabla con Liquid Clustering
spark.sql("""
    CREATE TABLE benchmark_liquid (
        transaction_id STRING,
        fecha DATE,
        cliente_id STRING,
        cuenta_id STRING,
        tipo_transaccion STRING,
        monto DECIMAL(18,2),
        region STRING,
        sucursal STRING,
        canal STRING,
        estado STRING
    )
    USING DELTA
    CLUSTER BY (fecha, cliente_id, region)
""")

# Insertar datos
df_benchmark.write.format("delta").mode("append").saveAsTable("benchmark_liquid")

# Ejecutar OPTIMIZE para aplicar clustering
spark.sql("OPTIMIZE benchmark_liquid")

print("✅ Tabla benchmark_liquid creada y optimizada")

In [ ]:
# Comparar tamaños y estructura de archivos
print("📊 COMPARACIÓN DE ESTRUCTURAS")
print("="*60)

for tabla in ["benchmark_partitioned", "benchmark_zorder", "benchmark_liquid"]:
    print(f"\n📦 {tabla}")
    
    # Obtener detalles
    detail = spark.sql(f"DESCRIBE DETAIL {tabla}").collect()[0]
    
    print(f"   Tamaño: {detail['sizeInBytes'] / (1024*1024):.2f} MB")
    print(f"   Archivos: {detail['numFiles']}")
    
    # Verificar propiedades de clustering
    props = spark.sql(f"SHOW TBLPROPERTIES {tabla}").collect()
    clustering_cols = None
    for prop in props:
        if 'clusteringColumns' in prop['key']:
            clustering_cols = prop['value']
            break
    
    if clustering_cols:
        print(f"   Liquid Clustering: {clustering_cols}")
    
    # Verificar particiones
    if detail['partitionColumns']:
        print(f"   Particiones: {detail['partitionColumns']}")

---
## 3. Benchmarks de Rendimiento

Ejecutamos las mismas consultas en las tres tablas para comparar:

In [ ]:
import time
from typing import List, Dict

def benchmark_query(query: str, tabla: str, num_runs: int = 3) -> Dict:
    """
    Ejecuta una query múltiples veces y retorna estadísticas.
    """
    times = []
    rows_scanned = 0
    
    for i in range(num_runs):
        # Limpiar cache
        spark.catalog.clearCache()
        
        # Ejecutar query
        start = time.time()
        result = spark.sql(query.format(tabla=tabla))
        count = result.count()  # Forzar ejecución
        end = time.time()
        
        times.append(end - start)
        rows_scanned = count
    
    return {
        'tabla': tabla,
        'min_time': min(times),
        'max_time': max(times),
        'avg_time': sum(times) / len(times),
        'rows': rows_scanned
    }

def run_benchmark_suite(queries: List[Dict], tablas: List[str]) -> None:
    """
    Ejecuta suite de benchmarks y muestra resultados.
    """
    for query_info in queries:
        print(f"\n{'='*70}")
        print(f"🔍 {query_info['name']}")
        print(f"   Query: {query_info['query'][:80]}...")
        print(f"{'='*70}")
        
        results = []
        for tabla in tablas:
            result = benchmark_query(query_info['query'], tabla)
            results.append(result)
            print(f"\n   📦 {tabla}")
            print(f"      Tiempo promedio: {result['avg_time']:.3f}s")
            print(f"      Rango: {result['min_time']:.3f}s - {result['max_time']:.3f}s")
        
        # Comparar
        best = min(results, key=lambda x: x['avg_time'])
        print(f"\n   🏆 Mejor: {best['tabla']} ({best['avg_time']:.3f}s)")

In [ ]:
# Definir queries de benchmark
benchmark_queries = [
    {
        'name': 'Query 1: Filtro por fecha específica',
        'query': "SELECT * FROM {tabla} WHERE fecha = '2024-06-15'"
    },
    {
        'name': 'Query 2: Filtro por rango de fechas',
        'query': "SELECT * FROM {tabla} WHERE fecha BETWEEN '2024-01-01' AND '2024-03-31'"
    },
    {
        'name': 'Query 3: Filtro por cliente específico',
        'query': "SELECT * FROM {tabla} WHERE cliente_id = 'CLI-001234'"
    },
    {
        'name': 'Query 4: Filtro combinado (fecha + región)',
        'query': "SELECT * FROM {tabla} WHERE fecha >= '2024-06-01' AND region = 'NORTE'"
    },
    {
        'name': 'Query 5: Agregación por fecha y región',
        'query': "SELECT fecha, region, SUM(monto) as total, COUNT(*) as txns FROM {tabla} WHERE fecha >= '2024-01-01' GROUP BY fecha, region"
    },
    {
        'name': 'Query 6: Filtro multi-columna (fecha + cliente + región)',
        'query': "SELECT * FROM {tabla} WHERE fecha >= '2024-06-01' AND cliente_id LIKE 'CLI-00%' AND region IN ('NORTE', 'SUR')"
    }
]

tablas_benchmark = [
    'benchmark_partitioned',
    'benchmark_zorder', 
    'benchmark_liquid'
]

print("🚀 INICIANDO BENCHMARKS")
print("   Cada query se ejecuta 3 veces para promediar\n")

run_benchmark_suite(benchmark_queries, tablas_benchmark)

In [ ]:
# Análisis de Data Skipping (archivos leídos)
print("\n📊 ANÁLISIS DE DATA SKIPPING")
print("="*60)
print("\nMenor número de archivos leídos = mejor data skipping\n")

test_query = "SELECT COUNT(*) FROM {tabla} WHERE fecha = '2024-06-15' AND cliente_id = 'CLI-001234'"

for tabla in tablas_benchmark:
    # Limpiar cache
    spark.catalog.clearCache()
    
    # Ejecutar con explain
    print(f"\n📦 {tabla}:")
    spark.sql(test_query.format(tabla=tabla)).explain("formatted")

---
## 4. Características Avanzadas de Liquid Clustering

### 4.1 Cambiar Clustering Keys (Sin Reescribir)

In [ ]:
# Una de las mayores ventajas: cambiar clustering keys dinámicamente
print("🔄 CAMBIO DINÁMICO DE CLUSTERING KEYS")
print("="*60)

# Ver clustering actual
print("\n1️⃣ Clustering actual:")
display(spark.sql("DESCRIBE DETAIL benchmark_liquid").select("clusteringColumns"))

# Cambiar clustering keys
print("\n2️⃣ Cambiando a CLUSTER BY (tipo_transaccion, canal, fecha)...")
spark.sql("""
    ALTER TABLE benchmark_liquid 
    CLUSTER BY (tipo_transaccion, canal, fecha)
""")

print("\n3️⃣ Nuevo clustering (sin reescribir datos):")
display(spark.sql("DESCRIBE DETAIL benchmark_liquid").select("clusteringColumns"))

print("\n✅ ¡Las keys se cambiaron instantáneamente!")
print("   Los datos existentes se re-clusterizan gradualmente con OPTIMIZE")

In [ ]:
# Aplicar OPTIMIZE para re-clusterizar con las nuevas keys
print("🔄 Aplicando OPTIMIZE para re-clusterizar...")

result = spark.sql("OPTIMIZE benchmark_liquid")
display(result)

print("\n✅ Datos re-clusterizados con las nuevas keys")

In [ ]:
# Restaurar clustering original para el proyecto
print("🔄 Restaurando clustering original...")
spark.sql("""
    ALTER TABLE benchmark_liquid 
    CLUSTER BY (fecha, cliente_id, region)
""")
spark.sql("OPTIMIZE benchmark_liquid")
print("✅ Clustering restaurado")

### 4.2 Eliminar Clustering (CLUSTER BY NONE)

In [ ]:
# Crear tabla temporal para demostrar
spark.sql("""
    CREATE OR REPLACE TABLE demo_clustering_removal
    CLUSTER BY (fecha, region)
    AS SELECT * FROM benchmark_liquid LIMIT 10000
""")

print("📦 Tabla con clustering:")
display(spark.sql("DESCRIBE DETAIL demo_clustering_removal").select("clusteringColumns"))

# Eliminar clustering
print("\n🗑️ Eliminando clustering...")
spark.sql("ALTER TABLE demo_clustering_removal CLUSTER BY NONE")

print("\n📦 Tabla sin clustering:")
display(spark.sql("DESCRIBE DETAIL demo_clustering_removal").select("clusteringColumns"))

# Limpiar
spark.sql("DROP TABLE demo_clustering_removal")
print("\n✅ Demo completada")

### 4.3 Convertir Tabla Existente a Liquid Clustering

In [ ]:
# Crear tabla sin clustering
print("📦 Creando tabla SIN clustering...")
spark.sql("""
    CREATE OR REPLACE TABLE demo_conversion
    AS SELECT * FROM benchmark_liquid LIMIT 50000
""")

print("\n   Estado inicial:")
display(spark.sql("DESCRIBE DETAIL demo_conversion").select("clusteringColumns"))

# Agregar Liquid Clustering a tabla existente
print("\n🔄 Agregando Liquid Clustering...")
spark.sql("""
    ALTER TABLE demo_conversion
    CLUSTER BY (fecha, cliente_id)
""")

print("\n   Después de ALTER:")
display(spark.sql("DESCRIBE DETAIL demo_conversion").select("clusteringColumns"))

# OPTIMIZE para aplicar clustering
print("\n🔄 Aplicando OPTIMIZE...")
spark.sql("OPTIMIZE demo_conversion")

print("\n✅ Tabla convertida exitosamente a Liquid Clustering")

# Limpiar
spark.sql("DROP TABLE demo_conversion")

---
## 5. Mejores Prácticas

In [ ]:
# Mejores prácticas documentadas
print("""
📚 MEJORES PRÁCTICAS PARA LIQUID CLUSTERING
══════════════════════════════════════════════════════════════════

1️⃣  SELECCIÓN DE COLUMNAS DE CLUSTERING
────────────────────────────────────────
   ✅ Elegir columnas que se usen frecuentemente en WHERE
   ✅ Priorizar columnas con alta cardinalidad
   ✅ Máximo 4 columnas (orden importa)
   ❌ Evitar columnas que cambian constantemente
   ❌ No usar columnas con muy baja cardinalidad (<100 valores)

   Ejemplo óptimo para transacciones financieras:
   CLUSTER BY (fecha, cliente_id, tipo_transaccion)
                 │          │              │
                 │          │              └─ Filtros por tipo
                 │          └─ Búsquedas por cliente
                 └─ Rangos de fecha (más común)

2️⃣  CUÁNDO USAR LIQUID CLUSTERING vs PARTITIONING
──────────────────────────────────────────────────
   
   LIQUID CLUSTERING es mejor cuando:
   ✅ Queries filtran por múltiples columnas
   ✅ Los patrones de consulta pueden cambiar
   ✅ Tablas < 10TB
   ✅ No quieres gestionar particiones manualmente
   
   PARTITIONING tradicional es mejor cuando:
   ✅ Siempre filtras por la misma columna (ej: fecha)
   ✅ Tablas muy grandes (>10TB)
   ✅ Necesitas eliminar particiones completas frecuentemente
   ✅ Tienes restricciones de retention por partición

3️⃣  MANTENIMIENTO Y OPTIMIZE
─────────────────────────────
   ✅ Ejecutar OPTIMIZE regularmente (diario o después de ETL)
   ✅ Usar Predictive Optimization si disponible (Unity Catalog)
   ✅ Monitorear con DESCRIBE HISTORY
   
   -- OPTIMIZE básico
   OPTIMIZE nombre_tabla
   
   -- OPTIMIZE con WHERE para tablas grandes
   OPTIMIZE nombre_tabla WHERE fecha >= current_date() - 7

4️⃣  NO MEZCLAR CON PARTITIONING
────────────────────────────────
   ❌ No usar PARTITION BY junto con CLUSTER BY
   ❌ Convertir tablas particionadas requiere reescritura

5️⃣  LÍMITES Y CONSIDERACIONES
──────────────────────────────
   • Máximo 4 columnas de clustering
   • Compatible con DBR 13.3+
   • Clustering manual requiere DBR 15.2+ (Standard tier)
   • Auto-clustering (Predictive Optimization) requiere Unity Catalog

══════════════════════════════════════════════════════════════════
""")

---
## 6. Aplicación a las Tablas del Proyecto

In [ ]:
# Revisar clustering en las tablas del proyecto
print("📊 LIQUID CLUSTERING EN TABLAS DEL PROYECTO")
print("="*60)

tablas_proyecto = [
    ("bronze_clientes", "CDC de clientes"),
    ("bronze_cuentas", "CDC de cuentas"),
    ("bronze_transacciones", "CDC de transacciones"),
    ("silver_dim_clientes", "Dimensión SCD Type 2"),
    ("silver_dim_cuentas", "Dimensión SCD Híbrida"),
    ("gold_fact_transacciones_diarias", "Fact Table"),
    ("gold_kpi_clientes", "KPIs de clientes"),
    ("gold_kpi_cuentas", "KPIs de cuentas"),
    ("gold_metricas_diarias", "Métricas diarias")
]

for tabla, descripcion in tablas_proyecto:
    try:
        detail = spark.sql(f"DESCRIBE DETAIL {tabla}").collect()[0]
        clustering = detail.get('clusteringColumns', [])
        
        print(f"\n📦 {tabla}")
        print(f"   └─ {descripcion}")
        if clustering:
            print(f"   └─ Clustering: {clustering}")
        else:
            print(f"   └─ Clustering: No configurado")
    except Exception as e:
        print(f"\n❌ {tabla}: No encontrada")

In [ ]:
# Función para optimizar todas las tablas del proyecto
def optimize_project_tables():
    """
    Ejecuta OPTIMIZE en todas las tablas del proyecto con Liquid Clustering.
    """
    print("🔄 OPTIMIZANDO TABLAS DEL PROYECTO")
    print("="*60)
    
    tablas = [
        "bronze_clientes",
        "bronze_cuentas", 
        "bronze_transacciones",
        "silver_dim_clientes",
        "silver_dim_cuentas",
        "gold_fact_transacciones_diarias",
        "gold_kpi_clientes",
        "gold_kpi_cuentas",
        "gold_metricas_diarias"
    ]
    
    for tabla in tablas:
        try:
            print(f"\n🔄 Optimizando {tabla}...")
            result = spark.sql(f"OPTIMIZE {tabla}")
            metrics = result.collect()[0]
            print(f"   ✅ Archivos reescritos: {metrics.get('numFilesAdded', 0)}")
        except Exception as e:
            print(f"   ⚠️ Error: {str(e)[:50]}")
    
    print("\n" + "="*60)
    print("✅ Optimización completada")

# Ejecutar optimización (comentado para no ejecutar automáticamente)
# optimize_project_tables()

---
## 7. Monitoreo de Clustering

In [ ]:
# Función para monitorear estado de clustering
def check_clustering_status(tabla: str):
    """
    Muestra el estado de clustering de una tabla.
    """
    print(f"\n📊 ESTADO DE CLUSTERING: {tabla}")
    print("="*60)
    
    # Detalles de la tabla
    detail = spark.sql(f"DESCRIBE DETAIL {tabla}").collect()[0]
    
    print(f"\n📦 Información General:")
    print(f"   Tamaño: {detail['sizeInBytes'] / (1024*1024):.2f} MB")
    print(f"   Archivos: {detail['numFiles']}")
    print(f"   Clustering Columns: {detail.get('clusteringColumns', 'N/A')}")
    
    # Historial de operaciones OPTIMIZE
    print(f"\n📜 Últimas operaciones OPTIMIZE:")
    history = spark.sql(f"""
        SELECT 
            version,
            timestamp,
            operation,
            operationMetrics.numAddedFiles as archivos_agregados,
            operationMetrics.numRemovedFiles as archivos_eliminados
        FROM (DESCRIBE HISTORY {tabla})
        WHERE operation = 'OPTIMIZE'
        ORDER BY version DESC
        LIMIT 5
    """)
    display(history)

# Ejemplo
check_clustering_status("benchmark_liquid")

---
## 8. Limpieza de Tablas de Benchmark

In [ ]:
# Limpiar tablas de benchmark (opcional)
cleanup = False  # Cambiar a True para eliminar

if cleanup:
    print("🗑️ Limpiando tablas de benchmark...")
    
    for tabla in ['benchmark_partitioned', 'benchmark_zorder', 'benchmark_liquid']:
        spark.sql(f"DROP TABLE IF EXISTS {tabla}")
        print(f"   ✅ {tabla} eliminada")
    
    print("\n✅ Limpieza completada")
else:
    print("ℹ️ Tablas de benchmark conservadas para análisis adicional")
    print("   Cambiar 'cleanup = True' para eliminar")

---
## 📌 Resumen: Liquid Clustering

### Ventajas Principales

| Característica | Beneficio |
|----------------|----------|
| **Flexibilidad** | Cambiar clustering keys sin reescribir datos |
| **Simplicidad** | No necesitas gestionar particiones |
| **Data Skipping** | Mejor rendimiento en queries con filtros |
| **Incremental** | OPTIMIZE solo procesa datos nuevos |
| **Compatible** | Funciona con CDC y Time Travel |

### Cuándo Usar

- ✅ Tablas con múltiples patrones de consulta
- ✅ Datos que crecen incrementalmente
- ✅ Cuando no estás seguro de las mejores particiones
- ✅ Tablas medianas (<10TB)

### Comandos Clave

```sql
-- Crear tabla con Liquid Clustering
CREATE TABLE mi_tabla (...) CLUSTER BY (col1, col2, col3);

-- Cambiar clustering keys
ALTER TABLE mi_tabla CLUSTER BY (nueva_col1, nueva_col2);

-- Eliminar clustering
ALTER TABLE mi_tabla CLUSTER BY NONE;

-- Aplicar/actualizar clustering
OPTIMIZE mi_tabla;
```

---
**Notebook 05 Completado** ✅